# Калибровки модели на open source моделе Mistral

Выполняется на kaggle с использованием GPU

In [ ]:
%%capture
!pip uninstall -y bitsandbytes peft transformers

# Установка совместимых версий для CUDA 12.1
!pip install -q torch==2.3.1+cu121 \
               torchvision==0.18.1+cu121 \
               torchaudio==2.3.1 \
               --extra-index-url https://download.pytorch.org/whl/cu121

!pip install -q bitsandbytes==0.45.0 \
               transformers==4.41.1 \
               peft==0.11.0 \
               accelerate==0.29.3

!pip install langgraph

In [ ]:
# Настройка окружения для CUDA 12.1
import os
os.environ["BNB_CUDA_VERSION"] = "121"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.1/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

In [ ]:
import numpy as np

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("token")
login(token = hf_token)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "IlyaGusev/saiga_mistral_7b_lora",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=dict(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
)

tokenizer = AutoTokenizer.from_pretrained(
    "IlyaGusev/saiga_mistral_7b_lora",
    use_fast=False
)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated

In [ ]:
class AgentState(TypedDict):
    """Состояние агента"""
    messages: Annotated[list, add_messages]
    user_query: str
    profile: str
    search_performed: bool
    search_results: str

class Agent_opensourceLLM:
    """LangGraph агент"""
    biases = ['стадность','излишняя самоуверенность']
    sex = {1: 'мужчина', 2: 'женщина'}
    tip = {1:'Москва или Санкт-Петербург',
       2: 'город - миллионник (численность людей)',
       3: 'городе ,где проживают 500-950 тыс. человек',
       4: 'городе ,где проживают 100-500 тыс. человек',
       5: 'городе ,где проживают до 100 тыс. человек',
       6: 'селе',
       7: 'неизвестно, что за город'}
    fo =  {1: 'Центральный федеральный округ',
       2: 'Северо-Западный федеральный округ',
       3: 'Южный федеральный округ',
       4: 'Северо-Кавказский федеральный округ',
       5: 'Приволжский федеральный округ',
       6: 'Уральский федеральный округ',
       7: 'Сибирский федеральный округ',
       8: 'Дальневосточный федеральный округ'}
    prof = {1: 'неработающий пенсионер',
       2: 'работающий пенсионер',
       3: 'неработающий учащийся, студент',
       4: 'работающий учащийся, студент',
       5: 'безработный',
       6: 'находящийся в декретном отпуске',
       7: 'работающий в найме',
       8: 'предприниматель',
       9: 'самозанятый',
       99: '-',
       999: '-'}
    dohod = {1:'очень хорошее',
       2: 'хорошее',
       3: 'среднее',
       4: 'плохое',
       5: 'очень плохое',
       99: 'неизвестно'}
    edu = {1:'неполное среднее образование',
       2: 'среднее образование (школа или ПТУ)',
       3: 'среднее специальное образование (техникум)',
       4: 'незаконченное высшее (с 3-го курса ВУЗа)',
       5: 'высшее образование',
       6: 'неизвестное образование',
       999: 'неизвестное образование'}
    def __init__(self, model, tokenizer, row):
        self.model = model
        self.tokenizer = tokenizer
        self.row = row        
        self.bias = np.random.choice(self.biases,1)[0]
        self.graph = self._create_graph()

    def _generate_text(self, system_prompt: str, user_prompt: str) -> str:
        # Форматируем промпт в формате Saiga/Mistral
        prompt = f"<s>system\n{system_prompt}</s>\n<s>user\n{user_prompt}</s>\n<s>bot\n"
    
        # Токенизация и генерация
        inputs = self.tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id
        )
    
        # Декодинг и очистка ответа
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True) 
        response = self._extract_generated_text(full_response)
        return response

    def _extract_generated_text(self, text: str) -> str:
        """Извлекает только сгенерированный текст (последний ответ бота)"""
        # Оставить только текст после последнего <s>bot\n
        if "<s>bot\n" in text:
            text = text.split("<s>bot\n")[-1]
        # Обрезать по первому </s>
        if "</s>" in text:
            text = text.split("</s>")[0]
        # Удалить все служебные теги
        text = text.replace("<s>", "").replace("</s>", "")
        text = text.replace("system\n", "").replace("user\n", "").replace("bot\n", "")
        # Оставить только последний абзац (если всё равно что-то не то)
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]
            
        if len(paragraphs) > 1:
            return '\n'.join(paragraphs[-2:])  # последние 2 
        return text.strip()

    def create_profile(self): 
        '''Создает промпт на основе demographics_info и bias'''
        profile_prompt = f"""Представь, что сейчас 2023 год, ТЫ {self.prof[self.row['PROF']]} {self.sex[self.row['SEX']]} {self.row['AGE']} лет, проживающий в России ({self.fo[self.row['FO']]}) в {self.tip[self.row['TIP']]}. Твое материальное состояние можно охарактеризовать, как {self.dohod[self.row['DOHOD']]}. Ты получил {self.edu[self.row['EDU']]}. Тебе присуща {self.bias}.    
        """
        return profile_prompt

    def _create_graph(self):
        """Создает LangGraph граф"""
        
        # Определяем граф
        graph_builder = StateGraph(AgentState)
        
        # Добавляем nodes
        graph_builder.add_node("initialize_profile", self._initialize_profile_node)
        graph_builder.add_node("search", self._search_node)
        graph_builder.add_node("generate_response", self._generate_response_node)
        
        # Добавляем edges
        graph_builder.add_edge(START, "initialize_profile")
        graph_builder.add_edge("initialize_profile", "search")  
        graph_builder.add_edge("search", "generate_response")
        graph_builder.add_edge("generate_response", END)
        
        return graph_builder.compile()

    def _initialize_profile_node(self, state: AgentState) -> AgentState:
        """Node для инициализации профиля"""
        profile = self.create_profile()
        return {
            **state,
            "profile": profile,
            "search_performed": False,
            "search_results": ""
        }
        
    def _search_node(self, state: AgentState) -> AgentState:
        """Node для поиска с обработкой ошибок"""
        search_query = self._create_search_query(state)
            
        return {
            **state,
            "search_performed": True,
            "search_results": search_query
            }

    def _create_search_query(self, state: AgentState) -> str:
        """Создает поисковый запрос"""
        query = f"""{state['profile']}\nОпиши текущую экономическую ситуацию для меня с учетом МОИХ демо-географических характеристик на момент 2023 года."""
        system_prompt = """Ты ИИ-ассистент, который помогает создать подробный портрет человека"""
        return self._generate_text(system_prompt, query)


    def _generate_response_node(self, state: AgentState) -> AgentState:
        system_prompt = f"""{state['profile']}\n {state['search_results']}\n 
        Ответь на вопрос от первого лица: {state['user_query']}.
        """
        response = self._generate_text(system_prompt, state['user_query'])
        response = self._extract_generated_text(response)
        return {
                **state,
            "messages": [AIMessage(content=response)]
        }

    def process_query(self, query: str) -> str:
        """Обрабатывает запрос пользователя через LangGraph"""
        try:
            # Инициализируем состояние
            initial_state = {
                "messages": [HumanMessage(content=query)],
                "user_query": query,
                "profile": "",
                "search_performed": False,
                "search_results": ""
            }
            
            # Запускаем граф
            result = self.graph.invoke(initial_state)
            
            # Возвращаем последнее сообщение
            if result.get("messages") and len(result["messages"]) > 0:
                return result["messages"][-1].content
            else:
                return "Извините, не удалось получить ответ."
                
        except Exception as e:
            return f"Ошибка при обработке запроса: {str(e)}"

